# Skin Cancer Classification Challenge

**Summary : ** skin cancer.

## Preliminaries and Imports

In [1]:
#general & system\n",
import os

#data processing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = 10, 10  #default setting
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

#data augmentation
from PIL import Image
from random import choice
import cv2
import numpy as np
#import keras.preprocessing.image as prep

#ML part
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image as kimage
from keras.models import Sequential
from keras.layers import GlobalMaxPooling2D, Dense, BatchNormalization, GlobalAveragePooling2D, Dropout,Activation,Flatten
from keras.models import Model
from keras.optimizers import SGD
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator

#validation
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

%matplotlib inline

Using TensorFlow backend.


## I. Load Data

- **Rq : ** Datasets are quite light, can be fully loaded in a laptop memory with ease.

In [2]:
%%time
meta = pd.read_csv("data/metadata.csv")

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 6.05 ms


In [3]:
%%time
filelist = os.listdir("data/resized/")
X_train = np.array([np.array(Image.open("data/resized/"+fname)) for fname in filelist])

CPU times: user 3.3 s, sys: 172 ms, total: 3.47 s
Wall time: 3.47 s


In [4]:
X_train.shape

(1900, 299, 299, 3)

In [5]:
#get target
filelist = [x.replace(".jpg","") for x in filelist]
strates = [meta[meta["name"]==x]["meta_clinical_benign_malignant"].values[0] for x in filelist]
Y_train = [1 if x=="malignant" else 0 for x in strates]

In [6]:
#control
print(filelist[0:5])
print(Y_train[0:5])
print(strates[0:5])

['ISIC_0000256', 'ISIC_0012768', 'ISIC_0012987', 'ISIC_0013676', 'ISIC_0013911']
[0, 0, 0, 0, 0]
['benign', 'benign', 'benign', 'indeterminate/benign', 'benign']


## II. Data Preparation

- **Resampling**

- **Class weights**

In [ ]:
class_weight = class_weight.compute_class_weight('balanced', np.unique(Y_train), Y_train)

- **Data splitting**

In [7]:
#data splitting
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train, random_state=42, train_size=0.80,
                                                      stratify = strates)

## III. Deep Learning with Transfer Learning

In [8]:
weights_path = 'model/checkpoints/'

if not os.path.exists(weights_path):
    os.makedirs(weights_path)
    
epochs = 500
batch_size = 10

- **Using VGG16 : **

In [9]:
#load base model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(299,299,3))
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 299, 299, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 299, 299, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 299, 299, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 149, 149, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 149, 149, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 149, 149, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 74, 74, 128)       0         
__________

In [10]:
#adding top layers
    #sequentialy
add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:])) #Flatten/GlobalAveragePooling2D
add_model.add(Dense(1024, activation='relu'))
add_model.add(Dropout(0.25))
add_model.add(Dense(512, activation='relu'))
add_model.add(Dropout(0.25))
add_model.add(Dense(1, activation='sigmoid'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 299, 299, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 299, 299, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 299, 299, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 149, 149, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 149, 149, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 149, 149, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 74, 74, 128)       0         
__________

In [11]:
#freeze lower layers of the model
#for layer in model.layers[:]:
    #layer.trainable = False
for layer in model.layers[0:]:
    layer.trainable = True
    
#compile
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
%%time
#Lets define the image transormations that we wan

gen = ImageDataGenerator(horizontal_flip=True,
                         vertical_flip=True,
                         width_shift_range=0.2,
                         height_shift_range=0.2,
                         zoom_range=0.2,
                         rotation_range=40)

# Here is the function that merges our two generators
# We use the exact same generator with the same random seed for both the y and angle arrays
def gen_flow_for_one_input(X1, y):
    genX1 = gen.flow(X1, y, batch_size=batch_size, seed=42)
    while True:
        X1i = genX1.next()
        yield X1i[0], X1i[1]

#Finally create out generator
gen_flow = gen_flow_for_one_input(X_train, Y_train)

from keras.callbacks import EarlyStopping, ModelCheckpoint
epochs_to_wait_for_improve = 50
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=epochs_to_wait_for_improve)
checkpoint_callback = ModelCheckpoint(weights_path + 'BestKerasModelVGG16_flat.h5', monitor='val_loss',
                                      verbose=1, save_best_only=True, mode='min')
#fit the model
model.fit_generator(gen_flow, validation_data=(X_valid, Y_valid),
                    steps_per_epoch=int(np.ceil(len(X_train)/batch_size)),
                    epochs=500, verbose=1, callbacks=[early_stopping_callback, checkpoint_callback])
                    #class_weights = class_weight

Epoch 1/500
152/152 [==============================] - 191s 1s/step - loss: 0.6863 - acc: 0.7487 - val_loss: 0.5668 - val_acc: 0.7737
Epoch 2/500
152/152 [==============================] - 194s 1s/step - loss: 0.5461 - acc: 0.7730 - val_loss: 0.5300 - val_acc: 0.7737
Epoch 3/500
152/152 [==============================] - 196s 1s/step - loss: 0.5339 - acc: 0.7730 - val_loss: 0.5208 - val_acc: 0.7737
Epoch 4/500
152/152 [==============================] - 193s 1s/step - loss: 0.5333 - acc: 0.7724 - val_loss: 0.5228 - val_acc: 0.7737
Epoch 5/500
152/152 [==============================] - 201s 1s/step - loss: 0.5225 - acc: 0.7737 - val_loss: 0.5273 - val_acc: 0.7737
Epoch 6/500
152/152 [==============================] - 205s 1s/step - loss: 0.5149 - acc: 0.7750 - val_loss: 0.4987 - val_acc: 0.7737
Epoch 7/500
152/152 [==============================] - 204s 1s/step - loss: 0.5116 - acc: 0.7684 - val_loss: 0.5113 - val_acc: 0.7816
Epoch 8/500
152/152 [==============================] - 202s 1s

Epoch 31/500
152/152 [==============================] - 149s 980ms/step - loss: 0.3839 - acc: 0.8211 - val_loss: 0.5118 - val_acc: 0.8132
Epoch 32/500
152/152 [==============================] - 149s 981ms/step - loss: 0.3836 - acc: 0.8276 - val_loss: 0.4660 - val_acc: 0.7789
Epoch 33/500
152/152 [==============================] - 149s 981ms/step - loss: 0.3851 - acc: 0.8164 - val_loss: 0.4974 - val_acc: 0.8053
Epoch 34/500
152/152 [==============================] - 152s 998ms/step - loss: 0.3795 - acc: 0.8217 - val_loss: 0.5485 - val_acc: 0.7868
Epoch 35/500
152/152 [==============================] - 150s 990ms/step - loss: 0.3687 - acc: 0.8336 - val_loss: 0.4903 - val_acc: 0.8000
Epoch 36/500
152/152 [==============================] - 150s 989ms/step - loss: 0.4105 - acc: 0.8053 - val_loss: 0.4699 - val_acc: 0.8132
Epoch 37/500
152/152 [==============================] - 151s 992ms/step - loss: 0.3527 - acc: 0.8388 - val_loss: 0.4692 - val_acc: 0.8000
Epoch 38/500
152/152 [============

ResourceExhaustedError: OOM when allocating tensor with shape[10,64,149,149]
	 [[Node: training/SGD/gradients/block1_pool/MaxPool_grad/MaxPoolGrad = MaxPoolGrad[T=DT_FLOAT, _class=["loc:@block1_pool/MaxPool"], data_format="NHWC", ksize=[1, 2, 2, 1], padding="VALID", strides=[1, 2, 2, 1], _device="/job:localhost/replica:0/task:0/device:GPU:0"](block1_conv2/Relu, block1_pool/MaxPool, training/SGD/gradients/block2_conv1/convolution_grad/Conv2DBackpropInput)]]

Caused by op u'training/SGD/gradients/block1_pool/MaxPool_grad/MaxPoolGrad', defined at:
  File "/home/paul/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/paul/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/paul/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/paul/anaconda2/lib/python2.7/site-packages/traitlets/config/applicatio

## IV. Evaluate Model

In [ ]:
model = load_model(weights_path+"BestKerasModelVGG16_flat.h5")
score = model.evaluate(X_valid, Y_valid, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
#compute log loss
pred = model.predict(X_valid)
log_loss(Y_valid,pred)